# Ejercicios de Preprocesamiento de datos

In [2]:
import math
from pyspark.sql.functions import *

1. ESTANDARIZACION

In [3]:
def ESTANDARIZACION(data):
    
    # Hallamos el valor de la desviacion estandar y media
    desviacion_estandar = data.stdev()
    media = data.mean()
    # Realizamos la estandarizacion de cada elemento, con map
    estandarizado = data.map(lambda x: (x-media)/(desviacion_estandar))
    # Retornamos la data normalizada
    return estandarizado

In [4]:
# Creamos nuestra data de un arreglo de numeros
data = [1,5,10,15,20,25,30,35,40,45,50]
# Volvemos este arreglo de números en un RDD de 3 particiones
dataRDD = sc.parallelize(data,3)
estandarizacion = ESTANDARIZACION(dataRDD)

# NORMALIZACION
estandarizacion.collect()

[-1.5374271566044004,
 -1.2821562324889528,
 -0.9630675773446432,
 -0.6439789222003337,
 -0.3248902670560242,
 -0.005801611911714657,
 0.3132870432325949,
 0.6323756983769044,
 0.951464353521214,
 1.2705530086655235,
 1.589641663809833]

2. ESCALONAMIENTO

In [5]:
def ESCALONAMIENTO(data):
    
    # Hallamos el valor maximo y minimo de la data
    maxval = data.max()
    minval = data.min()
    # Realizamos el escalonamiento de cada elemento, con map
    escalonado = data.map(lambda x: (x-minval)/(maxval-minval))
    # Retornamos la data normalizada
    return escalonado

In [6]:
# Creamos nuestra data de un arreglo de numeros
data = [1,5,10,15,20,25,30,35,40,45,50]
# Volvemos este arreglo de números en un RDD de 3 particiones
dataRDD = sc.parallelize(data,3)
escalonamiento = ESCALONAMIENTO(dataRDD)

# NORMALIZACION
escalonamiento.collect()

[0.0,
 0.08163265306122448,
 0.1836734693877551,
 0.2857142857142857,
 0.3877551020408163,
 0.4897959183673469,
 0.5918367346938775,
 0.6938775510204082,
 0.7959183673469388,
 0.8979591836734694,
 1.0]

3. NORMALIZACION

In [7]:
def NORMALIZACION(data):
    
    # Hallamos la norma a la data
    aux = data.map(lambda x:x**2).sum()
    norma = math.sqrt(aux)
    # Realizamos la normalización de cada elemento, con map
    normalizado = data.map(lambda x: x/norma)
    # Retornamos la data normalizada
    return normalizado

In [8]:
# Creamos nuestra data de un arreglo de numeros
data = [1,5,10,15,20,25,30,35,40,45,50]
# Volvemos este arreglo de números en un RDD de 3 particiones
dataRDD = sc.parallelize(data,3)
normalizacion = NORMALIZACION(dataRDD)

# NORMALIZACION
normalizacion.collect()

[0.010192414366433438,
 0.050962071832167194,
 0.10192414366433439,
 0.15288621549650158,
 0.20384828732866878,
 0.25481035916083594,
 0.30577243099300316,
 0.35673450282517033,
 0.40769657465733755,
 0.4586586464895047,
 0.5096207183216719]

4. TF

In [9]:
def TF(oraciones):
    
    # Tokenizamos cada oracion, y le agregamos una nueva variable para el conteo de apariciones
    mapeo = oraciones.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])

    # Agruparemos los pares key-value comunes, y agregaremos los valores de la misma para oftener sus apariciones
    reducir = mapeo.reduceByKey(lambda x,y:x+y)

    # APlicaremos la formula de tf para cada token , en cada documento
    tf = reducir.map(lambda x: (x[0][1], (x[0][0],(1 + math.log10(x[1]/(len(data[x[0][0]-1][1].split(" "))))))))
    reducir2 = reducir.map(lambda x: ((x[0][0],x[0][1]), (1 + math.log10(x[1]/(len(data[x[0][0]][1].split(" "))))) ))

    # Visualizacion de los tokens con sus valores TF
    muestra = reducir.map(lambda x: (x[0][1], "doc-"+str(x[0][0]),(1 + math.log10(x[1]/(len(data[x[0][0]][1].split(" ")))))))
    
    return tf, reducir2, muestra

In [10]:
# Creamos nuestra data de dos oraciones
data=[(0,"el es muy bueno en mineria de datos"),
      (1,"el sabe minerar datos con librerias de datos")]

# Volvemos este arreglo de tuplas en un RDD de 2 particiones
oraciones=sc.parallelize(data,2)
oraciones.collect()

# TF
tf = TF(oraciones)
tf[2].collect()

[('el', 'doc-0', 0.09691001300805646),
 ('bueno', 'doc-0', 0.09691001300805646),
 ('en', 'doc-0', 0.09691001300805646),
 ('mineria', 'doc-0', 0.09691001300805646),
 ('datos', 'doc-0', 0.09691001300805646),
 ('sabe', 'doc-1', 0.09691001300805646),
 ('minerar', 'doc-1', 0.09691001300805646),
 ('librerias', 'doc-1', 0.09691001300805646),
 ('de', 'doc-1', 0.09691001300805646),
 ('es', 'doc-0', 0.09691001300805646),
 ('muy', 'doc-0', 0.09691001300805646),
 ('de', 'doc-0', 0.09691001300805646),
 ('el', 'doc-1', 0.09691001300805646),
 ('datos', 'doc-1', 0.3979400086720376),
 ('con', 'doc-1', 0.09691001300805646)]

5. TF-IDF

In [11]:
def TF_IDF(reducir2, tf):
    
    # Asignamos los pares key-value a uno nuevo, el key será el token, value será TF y un auxiliar(1)
    mapeo2=reducir2.map(lambda x: (x[0][1],(x[0][0],x[1],1)))

    # Extraemos el token y auxiliar, esto nos ayudará a identificar la ocurrencia en las oraciones
    mapeo3=mapeo2.map(lambda x:(x[0],x[1][2]))

    # Reducimos por key, para obtener el conteo de apariciones en el documento
    reducir3=mapeo3.reduceByKey(lambda x,y:x+y)

    # Calcularemos el valor de IDF
    idf=reducir3.map(lambda x: (x[0],math.log10(1 + len(data)/x[1])))

    # RightOuterJoin entre valores tf y idf para unir el TF y IDF de cada token
    join = tf.rightOuterJoin(idf)

    # Multiplicamos los valores de TF y IDF para obtener TF-IDF
    tfidf = join.map(lambda x: (x[0],"doc-"+str(x[1][0][0]), x[1][0][1]*x[1][1]))

    return tfidf

In [12]:
# Creamos nuestra data de dos oraciones
data=[(0,"el es muy bueno en mineria de datos"),
      (1,"el sabe minerar datos con librerias de datos")]

# Volvemos este arreglo de tuplas en un RDD de 2 particiones
oraciones=sc.parallelize(data,2)
oraciones.collect()

# TF-IDF
tf = TF(oraciones) # Aplicamos tf
tfidf = TF_IDF(tf[1],tf[0]) # Aplicamos tfidf con el tf anterior
tfidf.collect()

[('el', 'doc-0', 0.0291728207956116),
 ('el', 'doc-1', 0.0291728207956116),
 ('datos', 'doc-0', 0.0291728207956116),
 ('datos', 'doc-1', 0.11979187908506812),
 ('con', 'doc-1', 0.04623782700130271),
 ('sabe', 'doc-1', 0.04623782700130271),
 ('minerar', 'doc-1', 0.04623782700130271),
 ('librerias', 'doc-1', 0.04623782700130271),
 ('es', 'doc-0', 0.04623782700130271),
 ('bueno', 'doc-0', 0.04623782700130271),
 ('en', 'doc-0', 0.04623782700130271),
 ('mineria', 'doc-0', 0.04623782700130271),
 ('de', 'doc-1', 0.0291728207956116),
 ('de', 'doc-0', 0.0291728207956116),
 ('muy', 'doc-0', 0.04623782700130271)]